In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import randint,uniform

# General Parameters

In [2]:
import time

# Step 1: Create a population

Individuals are defined by `['state', 'home_location', 'network_location', 'current_location' ]`, for example: 

`[0, 0, 4, 0]` is an individual who is susceptible, whose home is `0` and whose work is `4`, and who is currently at home.

In [3]:
n_pop = 10000
n_loc = 2601
n_net = 200


labels = [0,1,2,3,4]

person_attr = 4  # 4 attributes for a person: state, home, network, current location


transmit_rate = 0.2
e_to_i_rate = 0.2
recovery_rate = 0.1
death_fraction= 0.01
background_infection_rate = 0.01

random_test_rate =0.1
test_sensitivity =0.99
finish_test_rate =1.0

def create_person(state, home):
    net = randint(0,n_net)
    return np.array([state, home , net , home ]) # Create a person in state 'state', initially at home


pop = np.zeros((n_pop,person_attr),int)          # Empty population
link_weight = np.zeros(n_pop,float)              # Array for link weights
is_confined = np.zeros(n_pop,bool)               # Boolean array, is confined?
being_tested = np.zeros(n_pop,bool)              # Boolean array, is being tested?

n_per_location = np.zeros((n_loc+n_net,          # Array to hold total number of individuals per state 
                           len(labels)),int)     # for all locations: homes and networks

# Initial populations ###########

n_inf = 10
n_exp = 0
n_rec = 0
n_dead= 0
n_sus = n_pop - n_inf

output = np.array([0,n_sus, n_exp, n_inf, n_rec, n_dead],int)  # Output of the form t,S,E,I,R,D

##################################

for i in range(0,n_inf):                         # Set 10 infected person  with random network
    pop[i] = create_person(2,i)

    n_per_location[  pop[i][3] ][2] +=  1        # Increment infected in each person's location

    link_weight[i] = 0.1

for i in range(n_inf,n_pop):
    home = randint(0,n_loc)                      # Assign random homes for remainder with random network
    pop[i] = create_person(0,home)

    n_per_location[  pop[i][3] ][0] +=  1        # Increment susceptibles in each person's location

    link_weight[i] = 0.1


In [4]:
tf = 100.0

t = 0.0
t_out = 0

n_inf = 10
n_exp = 0
n_rec = 0
n_dead= 0
n_sus = n_pop - n_inf


while(t<tf):#n_exp+n_inf>0):
    #start = time.time()
    if(t>=t_out):
        output = np.vstack((output,[t_out,n_sus,n_exp,n_inf,n_rec,n_dead]))
        t_out += 1
        print(t)

    a0 = 0
    
    for i in range(0,n_pop):               # For every person
        if(pop[i][0]==0):                  # If they are susceptible S-> E
            n_infected_in_location = n_per_location[ pop[i][3] ][2] # Number of infected in person's current location
            a0 += transmit_rate * n_infected_in_location + background_infection_rate
    a1 = a0

    for i in range(0,n_pop):               # For every person
        if(pop[i][0]==1):                  # If they are exposed E-> I
            a0 += e_to_i_rate
    a2 = a0

    for i in range(0,n_pop):               # For every person
        if(pop[i][0]==2):                  # If they are infected I -> R // D
            a0 += recovery_rate
    a3 = a0

    for i in range(0,n_pop):               # For every person
        if(not is_confined[i] and pop[i][0]!=4): # If they aren't confined or dead
            a0 += random_test_rate
    a4 = a0

    for i in range(0,n_pop):               # For every person
        if(being_tested[i]):               # If they are being tested
            a0 += finish_test_rate
    a5 = a0

    # Shift people in network

    for i in range(0,n_pop):                # For every person

        if (not is_confined[i] and pop[i][0]!=4): # If they aren't confined or dead
            a0 += link_weight[i]                  # move them to their other location

#     print(a1)
#     print(a2)
#     print(a3)
#     print(a4)
#     print(a5)
#     print('----')
    
    
    dt = -np.log(uniform(0.0, 1.0)) / a0
    t = t + dt  

    a = uniform(0.0,1.0) * a0                     # Choose which process occurs

    k = 0
    
#     end = time.time()
#     print(end-start)

    if(a<a1):  # In this case, an S-> E
        psum = 0
        for i in range(0,n_pop):
            if pop[i][0] == 0:
                n_infected_in_location = n_per_location[ pop[i][3] ][2]
                psum += transmit_rate * n_infected_in_location + background_infection_rate

                if (psum>a):
                    k=i
                    break

        pop[k][0]=1                                # Set that person to exposed
        n_exp = n_exp + 1                          # Increment number of exposed
        n_sus = n_sus - 1                          # Decrement number of susceptible

        n_per_location[pop[k][3]][1] += 1          # Increment exposed in location of person
        n_per_location[pop[k][3]][0] -= 1          # Decrement susceptibles in location of person



    elif(a<a2): # Here E-> I
        psum = a1

        for i in range(0,n_pop):
            if(pop[i][0]==1):
                psum += e_to_i_rate

                if (psum>a):
                    k=i
                    break;

        pop[k][0]=2
        n_inf = n_inf + 1                          # Increment number of infected
        n_exp = n_exp - 1                          # Decrement number of exposed

        n_per_location[pop[k][3]][2] += 1          # Increment infected in location of person
        n_per_location[pop[k][3]][1] -= 1          # Decrement exposed in location of person




    elif(a<a3): # I -> R || D
        psum = a2

        for i in range(0,n_pop):

            if(pop[i][0]==2):
                psum += recovery_rate

                if(psum>a):
                    k=i
                    break

        is_confined[k] = True


        if(uniform(0.0,1.0)<death_fraction):           # Testing if person died or recovered
            pop[k][0]=4                                # Oh no! Dead!

            n_dead= n_dead+ 1                          # Increment number of recovered
            n_inf = n_inf - 1                          # Decrement number of infected

            n_per_location[pop[k][3]][4] += 1          # Increment dead in location of person
            n_per_location[pop[k][3]][2] -= 1          # Decrement infected in location of person

        else:
            pop[k][0]=3                                # Yay! Recovered!

            n_rec = n_rec + 1                          # Increment number of recovered
            n_inf = n_inf - 1                          # Decrement number of infected

            n_per_location[pop[k][3]][3] += 1          # Increment recovered in location of person
            n_per_location[pop[k][3]][2] -= 1          # Decrement infected in location of person


    elif(a<a4): # (Random) Test                                       
        psum = a3

        for i in range(0,n_pop):
            if(not is_confined[i] and pop[i][0]!=4):  # If the person isn't confined or dead
                psum += random_test_rate
                if(psum>a):
                    k=i
                    break

        being_tested[k]=True

    elif(a<a5):
        psum = a4

        for i in range(0,n_pop):

            if being_tested[i] == True:
                psum += finish_test_rate

                if(psum>a):
                    k=i
                    break

        being_tested[k]=False    

        if (pop[k][0]==2 and uniform(0.0,1.0)<test_sensitivity): # If the person is infected and the test works
            is_confined[k] = True                                # They are confined

            n_per_location[pop[k][3]][2] -= 1          # Decrement infected in current (NET or HOME) location of person
            pop[k][3] = pop[k][1]                                 # Send them home
            n_per_location[pop[k][3]][2] += 1          # Increment infected in HOME location of person


    else:  # All else failing, move people around
        psum = a5

        for i in range(0,n_pop):

            if (not is_confined[i] and pop[i][0]!=4): # If they aren't confined or dead
                psum += link_weight[i]                  # move them to their other location

                if(psum>a):
                    k=i
                    break

        if(pop[i][3]==pop[i][1]):                   # If the person is at home
            pop[i][3] = pop[i][2]                   # move them to their network location
        else:
            pop[i][3] = pop[i][1]                   # otherwise move them home
    

    # WHILE Loop ended
   

0.0
1.000246351654696
2.0000251455928
3.0005224426206585
4.001333203024495
5.000329132016326
6.000247707441079
7.00001556690977
8.000608137831046
9.00021652650874
10.000088546205776
11.000086866201217
12.000022355536029
13.000260895919222
14.0006613682372
15.00020593269592
16.000152742541186
17.000084090032516
18.000080849937696
19.000472091647573
20.000292682489636
21.00091358907149
22.000302943157497
23.00070051800014
24.000602871145855
25.000157327118995
26.000335178054993
27.000160392216916
28.000691182856336
29.0001227653468
30.000431891311447
31.000906661084166
32.00173378250599
33.00038705407537
34.00075464262148
35.00036623153485
36.00085063966122
37.00109612604661
38.001880269685316
39.00001524987533
40.00062632727915
41.000838446812566
42.00199616563605
43.00361558839839
44.000470564446275
45.00255118721296
46.00038371378654
47.00062936592366
48.0081180831075
49.003489964376165
50.009771435336134
51.001147060193965
52.00033971608738
53.002144822773204
54.00021125410601
55.001

In [6]:
np.savetxt("sample_v2.txt",output,delimiter=" ")